In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os, sys
lib_path = os.path.abspath(os.path.join('../classifier/'))
sys.path.append(lib_path)
import classifier

In [4]:
import nltk
import torch
import torchtext.vocab as vocab
import torch.optim as optim
import torch.autograd as autograd
import torch.nn as nn
import torch.utils.data as torchdata

In [5]:
import pickle

In [6]:
glove = vocab.GloVe(name='6B', dim=50)

# preprocess data

In [7]:
subset_train = .1

In [8]:
datapath = '../data/'

In [9]:
train = pd.read_csv(datapath + 'train.csv')
test = pd.read_csv(datapath + 'test.csv')
sub = pd.read_csv(datapath + 'sample_submission.csv')
train = train.iloc[:int(len(train) * subset_train), :]
test = test.iloc[:int(len(test) * subset_train), :]
test = test.fillna('unknown')
comment_types = train.columns[2:].values

In [10]:
lang = classifier.Language()

In [11]:
lang.process_sentences(train.comment_text.values,
                      test_sentences=list(test.comment_text.values))

In [12]:
init_embedding = classifier.create_init_embedding(lang, glove)

In [14]:
model = classifier.NER_LSTM(lang.vocab_size,
                           embedding_dim=50,
                           hidden_dim=64,
                            linear_dim=128, 
                           n_layers=1,
                            dropout=.3,
                            gpu=False,
                           init_embedding=init_embedding,
                           bidirectional=True)

In [15]:
loss = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#model = model.cuda()

In [17]:
trainer = classifier.Trainer(model,
                             optimizer,
                             loss,
                             lang.encoded_sentences,
                             train.loc[:, comment_types].values.astype(int),
                             batch_size = 256, gpu=False)

In [20]:
trainer.train(2)

Epoch: 1    loss: 0.2998, val: 0.1438
Epoch: 2    loss: 0.2841, val: 0.1230


In [21]:
X_test = lang.encoded_test_sentences
y_empty = torch.zeros(X_test.shape[0])
dataset = torchdata.TensorDataset(torch.LongTensor(X_test.astype(int)), y_empty)
loader = torchdata.DataLoader(dataset, batch_size = 256)
preds = []
model.eval()
for X, _ in loader:
    X = autograd.Variable(X).cuda()
    log_probs = model(X).cpu().data.numpy()
    preds.append(log_probs)
final = np.vstack(preds)

In [22]:
sub.loc[:, comment_types] = final
sub.to_csv(datapath + 'submission.csv', index = False)